In [46]:
# 絶対値最小剰余の計算
# Input：整数 𝑎, 𝑛 (𝑛>0)
# Output：𝑎 を 𝑛 で割った時の絶対値最小剰余
def mod(a, n):
    r = a % n
    if r > n/2:
        return r-n # 戻り値を指定
    else:
        return r # 戻り値を指定

In [47]:
# ランダムな多項式の生成
# Input：𝑘, 𝑝
# Output： ℤ/𝑝ℤ[𝑋]/⟨Φ_2n ⟩ (𝑛=2^𝑘 ) の一様ランダムな多項式の係数のリスト
#   (係数の範囲が (−𝑝/2,𝑝/2]内の整数であり，次数が 𝑛 未満の多項式の係数のリストを出力する）

import random, math
def random_poly(k, p):
    N = 1<<k # N=2^k (2のk乗)
    return [random.randint(int(-p/2)+1, int(p/2)) for _ in range(N)] # リスト内包表記
    # 残りを実装

In [48]:
# 多項式のリダクション
# Input：𝑘, 𝑝, 多項式の係数のリスト
# Output：入力多項式と⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_red(f, k, p):
    N =1<<k # N=2^k (2のk乗)
    if N > len(f)-1: # N=2^k > deg(f) なら係数だけを mod して出力
        return [mod(f[i], p) for i in range(len(f))]
    else:
        q, r = divmod(len(f)-1, N) # def(f) = len(f)-1. deg(f) を N=2^k で割った時の商 q と余り r を取得
        # fの係数 f[i] を更新する 
        while len(f) -N > 0:
            f[-N-1] = mod(f[-N-1] - f[-1], p)
            f = f[:-1]
        
        return [mod(f[i], p) for i in range(N)] # 更新した f[i] に mod関数を適用

In [49]:
# 2つの多項式の加法
# 入力：𝑘, 𝑝, 多項式 𝑓, 𝑔 の係数のリスト
# 出力： 𝑓+𝑔 と ⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_add(f, g, k, p):
    add = [] # f+g の係数を入れるためのリストを準備
    if len(f) < len(g): # 係数のリストに0を足して、見かけ上はf, gの次数を等しくする
        for i in range(len(g)-len(f)):
            f.append(0)
    elif len(f) > len(g):
        for i in range(len(f)-len(g)):
            g.append(0)
    for i in range(len(f)):
        add.append(mod(f[i]+g[i], p))
             # g の次数 i の係数 g[i] を f の次数 i の係数に足して mod 計算 
    return poly_red(add, k, p)

In [50]:
# 2つの多項式の乗法
# 入力：𝑘, 𝑝, 多項式 𝑓, 𝑔 の係数のリスト
# 出力： 𝑓𝑔 と ⟨Φ_2n ⟩ を法として合同な 次数が 𝑛 未満の多項式の係数のリスト

def poly_mult(f, g, k, p):
    if len(f) < len(g): # deg(f) < deg(g)=> fとgを入れ替える（簡単化のため deg(f)≧deg(g)とする. fg = gf より入れ替えても問題ない）
        # deg(f) < deg(g)=> fとgを入れ替える操作を追加（簡単化のため deg(f)≧deg(g)とする. fg = gf より入れ替えても問題ない）
        tmp = f
        f = g
        g = tmp
    mult = [] # fg の係数を入れるためのリストを準備
    for d in range(len(g)): # 次数が0～deg(g)までの係数を計算
        coef = 0
        for i in range(d+1):
            coef += f[i]*g[d-i]
        mult.append(mod(coef, p))
            # 係数を計算して mod して mult に格納
    
    for d in range(len(g), len(f)): # 次数がdeg(g)+1～deg(f)までの係数を計算(deg(f)=deg(g)の時は実行されない)
        coef = 0
        for i in range(len(g)):
            coef += f[d-i]*g[i]
        mult.append(mod(coef, p))
            # 係数を計算して mod して mult に格納
    
    # 次数が deg(f)+1～deg(fg)までの係数を計算
    for d in range(len(f), len(f) + len(g) - 1): # deg(f) = len(f)-1, deg(g) = len(g)-1 => deg(fg) = deg(f)+deg(g)=len(f)+len(g)-2
        coef = 0
        for i in range(d-len(f)+1, len(g)): # d-len(f)+1=d-(len(f)-1)=d-deg(f)
            # 係数を計算して mod して mult に格納
            coef += f[d-i]*g[i]
        mult.append(mod(coef, p))
    
    return poly_red(mult, k, p)

In [51]:
# 関数のテスト
k = 3
p = 19
f = [1,2,3,2,3,1,3,1] # f[i] は f の次数 i の係数
g = [3,2,3,4,3,3,6,7] # g[i] は g の次数 i の係数
print("fg (mod Φ2n)=", poly_mult(f, g, k, p))
print("f+g (mod Φ2n)=", poly_add(f, g, k, p))

fg (mod Φ2n)= [-2, 5, 6, 2, 5, 9, 4, 1]
f+g (mod Φ2n)= [4, 4, 6, 6, 6, 4, 9, 8]


In [52]:
k = 3
p = 19
f = [1,2,7,2,9,11,3,5]
g = [3,2,1,4,8,3,2,3]
print("fg (mod Φ2n)=", poly_mult(f, g, k, p))
print("f+g (mod Φ2n)=", poly_add(f, g, k, p))

fg (mod Φ2n)= [0, 3, 1, 4, 0, 5, 2, -2]
f+g (mod Φ2n)= [4, 4, 8, 6, -2, -5, 5, 8]


In [53]:
k = 3
p = 19
f = [1,2,7,2,9,11,3]
g = [3,2,1,4,8,3,2,3]
print("fg (mod Φ2n)=", poly_mult(f, g, k, p))
print("f+g (mod Φ2n)=", poly_add(f, g, k, p))

fg (mod Φ2n)= [-9, 8, 2, 6, -4, -4, -2, 2]
f+g (mod Φ2n)= [4, 4, 8, 6, -2, -5, 5, 3]


In [54]:
k = 3
p = 19
f = [1,2,7,2,9,11]
g = [3,2,1,4,8,2,3]
print("fg (mod Φ2n)=", poly_mult(f, g, k, p))
print("f+g (mod Φ2n)=", poly_add(f, g, k, p))

fg (mod Φ2n)= [-5, -9, -4, -7, -3, 4, 7, 7]
f+g (mod Φ2n)= [4, 4, 8, 6, -2, -6, 3]
